# EVM Symbolic Execution

Our goal is to add symbolic execution to a rust based implementation of the EVM.

## Symbolic execution in the large
Symbolic execution is a general (not EVM specific) program analysis technique.
- [Symbolic Execution Introduction](https://en.wikipedia.org/wiki/Symbolic_execution) read the intro paragraph and the example.
- [Symbolic Execution Lecture](https://www.youtube.com/watch?v=yRVZPvHYHzw) watch the whole thing. You should follow most all of it before you continue.

Symbolic execution also has applications to the EVM. It can be used to find conditions that lead to assertion violations as well as other use cases. 
- [HEVM Symbolic Execution](https://fv.ethereum.org/2020/07/28/symbolic-hevm-release/) read the full post but do not attempt to understand everything.
- [Dapptools testing integration](https://youtu.be/N9pJ9JieX10?t=990) Watch the linked timestamp to see how dapptools uses hevm to symbolically search for assertion failures

All symbolic execution implementations rely on an SMT Solver. You should have a basic understanding of the SMT Solver's role from the Symbolic Execution Lecture. We will be using Z3 for our examples because it has a nice python library. For our actual implementation we will be as solver agnostic as possible.
- [Z3 tutorial](https://theory.stanford.edu/~nikolaj/programmingz3.html) Read all of section 2 and sections 3.1-3.4. We will not be too deep in the weeds of the solver so focus on intuition instead of formal understanding. Feel free to play with some of the code samples.

## Vocabulary
- Concrete value - A value that is known at runtime. When concrete values are combined with other concrete values, the result is also concrete.
- Symbolic value - A value that is not known at runtime. Symbolic values still have known and fixed types. When symbolic values are combined with both concrete and symbolic values, the result is also symbolic.
- Memory model - How memory is represented so that it can hold symbolic values. "Memory" in the general sense of all readable and writable storage locations. I.e. for the EVM, the memory model refers to memory, storage, calldata, and code.
- Concrete instruction - An instruction that only operates on concrete values.
- Symbolic Instruction - An instruction that can operates on symbolic values, concrete values, and combinations of the two.
- (Path) Constraint set - The set of constraints collected while executing the program. They constrain the set of possible concrete values that the symbolic values can represent.
- Machine state - Abstract term referring to the current state of (a single) execution of the program. This includes the state of the memory mode, the constraint set, program counter, etc...
- Interpreter - The function that takes in one machine state, executes an instruction, and returns multiple machine states. If the returning multiple machine states part confuses you, that's ok, assume it returns a single updated machine state for now.
- Runtime - Abstract term referring to the code orchestrating the memory model, interpreter, and machine state to compute.
- Solver - The SMT solver that is used to check if constraints can be satisfied.
- Satisfied - There exists a possible solution to the set of constraints given. A solution can be used to replace symbolic values with concrete values that would lead to the current machine state.
- Unsatisfied - The solver could not find a solution to the set of constraints given. An unsatisfied set of constraints is used to assume that the current machine state cannot exist and it should not be explored further. Note that this doesn't mean that one doesn't exist and can be a false negative.


## Symbolic execution in the small
We are going to walk through some examples of implementing symbolic execution against an EVM-like assembly language. 

All symbolic execution implementations follow a common pattern of taking an existing program runtime that executes over concrete values and modifying it such that...
- All concrete readable and writable values are replaced with a memory model that handles symbolic values.
- The interpreter executes symbolic instructions
- The machine state tracks path constraints
- The runtime can pass a set of path constraints to a solver to check for satisfiability
- The Runtime can handle multiple concurrent machine states (ok if you don't understand why right now)

We are going to work our way up from simpler machines to more complex machines adding incremental symbolic execution functionality as needed.

One important thing to note is that there is not one way to implement symbolic execution and symbolic execution is not all or nothing. You can force parts of the runtime to only execute over concrete values and in fact frequently we do for performance reasons.

Note that all python code runs on Python 3.10 and we use the library [z3-solver](https://pypi.org/project/z3-solver/) to interact with Z3.

### Arithmetic operations on stack values
Our first example will be assuming that our language has ADD and SUB instructions that operate on the stack. For simplicity sake, we will assume that we can store actual integers on the stack and not fixed width bit representations of integers. We consider the "result" of the program to be the item on the top of the stack when the program ends. Take this simple implementation.

In [1]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int

class StackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We can compute things!

In [2]:
# (3 + 2) - 1 -> 4
m = StackArith([PUSH(1), PUSH(2), PUSH(3), ADD(), SUB()])
m.run()

4

Now what happens when we put symbolic values on the stack. Note that we don't have to change the implementation. This is because the z3 objects implement their own `+` method.

In [3]:
from z3 import *
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB()])
m.run()

z + y - x

The result of the computation is also symbolic. Note that the z3 library prints a nice string representation of the compound symbolic value. The actual type of the python value is from the Z3 package and the sort is still int. "Sorts"  are the Z3 equivalent of type (is this actually true?). Really "sort" comes from smtlib and not Z3 but that's getting ahead of ourselves.

In [4]:
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB()])
res = m.run()
print(res.__class__)
print(res.sort())

<class 'z3.z3.ArithRef'>
Int


We can also include concrete values on the stack, so the returned value contains both symbolic and concrete values.

In [5]:
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(3), ADD(), SUB()])
m.run()

3 + y - x

Just computing on symbolic values can be helpful in certain circumstances, but we want to be able to ask the question "what inputs do I have to pass this program for it to give this output". Let's add an additional instruction `ASSERT(n)` that adds the constraint that the top item on the stack is equal to its argument.

In [6]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class ASSERT:
    n: int

class SymStackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX We add the set of constraints to the machine state
        # Solver is the object from the z3 package that can be used
        # to hold constraints
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case ASSERT(n):
                # XXX We add the constraint to the machine state.
                self.constraints.add(self.stack[-1] == n)
            case _:
                assert False
            
    # Returns: (The symbolic value, the concrete value (if satisfiable), the model (if satisfiable))
    def run(self):
        while self.pc < len(self.code):
            self.step()
            
        # XXX We check if the set of constraints is satisfiable.
        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)
            return rv, eval_rv, model
        else:
            # The set of constraints is not satisfiable. There is no
            # model or concrete value to return
            return rv, None, None

In [7]:
m = SymStackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB(), ASSERT(5)])
m.run()

(z + y - x, 5, [y = 0, x = 0, z = 5])

The answer here is not terribly interesting, but it tells us that replacing x, y, and z with 0, 0, and 5 respectively will result in the final value on the stack being equal to 5. We can confirm this by re-running the same machine with the concrete values.

In [8]:
m = SymStackArith([PUSH(0), PUSH(0), PUSH(5), ADD(), SUB()])
m.run()

(5, 5, [])

What happens when the solution is not satisfiable? Let's try a basic example where we assert one symbolic value to be equal to two different concrete values

In [9]:
m = SymStackArith([PUSH(Int('x')), ASSERT(1), ASSERT(2)])
m.run()

(x, None, None)

The constraint solver tells us that no possible concret value of `x` can satisfy both constraints.

### Read only memory

Our next example will be adding readonly memory to the machine. In the concrete machine, memory will be represented with a python list that is passed into the machine. The machine will not modify the list. We will add an `MLOAD` instruction to read from the memory.

In [10]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

class ReadonlyMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                self.stack.append(self.memory[self.stack.pop()])
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

In this modified example, we now put the initial operand in memory and load it from memory before performing the addition.

In [11]:
mem = [0 for x in range(5)]
mem[4] = 3
# (mem[4] + 2) - 1
# -> (3 + 2) - 1 
# -> 4
m = ReadonlyMem([PUSH(1), PUSH(2), PUSH(4), MLOAD(), ADD(), SUB()], mem)
m.run()

4

Converting stack based arithmetic operations to symbolic was rather straight forward. They are operations, they take operands. In order to pass symbolic operands instead of concrete operands, the stack has to be able to hold symbolic values. 

Symbolic memory is more complicated because the memory can hold symbolic values, the memory can be indexed at symbolic values, and even the length of memory can be symbolic. This can be hard to conceptualize so we will incrementally add symbolic features to concrete memory. This is a common theme when figuring out how to implement symbolic execution. We start with adding simple symbolic features and incrementally add more symbolic features.

The simplest symbolic feature we can add is letting memory store symbolic values. Because the only major change from `ReadonlyMem` is that the stack and memory can hold symbolic values, we add an assertion that the memory index is concrete in `MLOAD`.

In [12]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class ASSERT:
    n: int

class SymReadonlyMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                idx = self.stack.pop()
                assert isinstance(idx, int), 'MLOAD: requires concrete index'
                self.stack.append(self.memory[idx])
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

Let's modify the previous example to now read a symbolic value from memory.

In [13]:
mem = [0 for x in range(5)]
mem[4] = Int('z')
m = SymReadonlyMem([PUSH(1), PUSH(2), PUSH(4), MLOAD(), ADD(), SUB(), ASSERT(4)], mem)
m.run()

(z + 2 - 1, 4, [z = 3])

We now get runtime errors when we try to read memory at a symbolic index

In [14]:
mem = [0 for x in range(5)]
m = SymReadonlyMem([PUSH(Int('x')), MLOAD()], mem)
try:
    m.run()
except AssertionError as e:
    print(e)

MLOAD: requires concrete index


If we want to read from memory at symbolic indices, we can't use a python list to represent memory. We have to use an SMT solver primitive called an uninterpreted function. Re-read [this section](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-euf--equality-and-uninterpreted-functions) if you need a refresher. The tl;dr is that uninterpreted functions can be used to model constraints on [mathematical functions](https://en.wikipedia.org/wiki/Function_(mathematics)) without worrying about the internals of how the function might be implemented. (TODO this could probably use some work) 

Because the uninterpreted function has to be defined over its entire domain, and the domain is the entire set of integers, we are saying that this machine has unbounded memory. We'll solve that later.

In [15]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class ASSERT:
    n: int

class SymReadonlyMemSymIndex:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX
        self.memory = Function('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                self.stack.append(self.memory(self.stack.pop()))
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

How to read output from the constraint solver for an uninterpreted function. For the following example, `f=[1->2, else -> 0]` can be read as `f(1)` returns `2` and for all other values in the input set, `f` returns 0. Note that this implicitly covers the first added constraint `f(0) == 0`. If you don't explicitly see a constraint you added in the output, check if the "else" covers it.

In [16]:
f = Function('f', IntSort(), IntSort())
s = Solver()
s.add(f(0) == 0)
s.add(f(1) == 2)
s.check()
s.model()

[f = [1 -> 2, else -> 0]]

We can execute the same example as before where we read the first operand from memory index 4, but this time we don't have to explicitly set that index in memory to being a symbolic value.

Note that the model says `memory = [else -> 3]`. Assume that memory is of size 5, this means memory would look like `[3, 3, 3, 3, 3]`. In the concrete machine, we set memory to  `[0, 0, 0, 0, 3]`. Both are valid initial machine states for the code we executed. The only thing that matters is that `memory[4] == 3`. The constraint solver is not guaranteed to give you back a particular valid machine state, just _a_ valid machine state. (TODO is "machine state" the best word to use in this context)

In [17]:
m = SymReadonlyMemSymIndex([
    PUSH(1), 
    PUSH(2), 
    PUSH(4), MLOAD(), 
    ADD(), 
    SUB(), 
    ASSERT(4)
])
m.run()

(memory(4) + 2 - 1, 4, [memory = [else -> 3]])

Let's now read from a symbolic memory index. The model resolves to the same initial memory state but says we read from index 0. Again, this is a _different_ initial machine state, but it is a _valid_ initial machine state. We just have to read the value 3 from a memory index.

In [18]:
m = SymReadonlyMemSymIndex([
    PUSH(1),
    PUSH(2),
    # XXX
    PUSH(Int('idx')), MLOAD(),
    ADD(),
    SUB(),
    ASSERT(4)
])
m.run()

(memory(idx) + 2 - 1, 4, [idx = 0, memory = [else -> 3]])

### Read/Write memory

Our next example takes the read only memory and adds writing. We will add an `MSTORE` instruction to write to memory.

Modifying the concrete machine to write to memory is straight forward.

In [19]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

class RWMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                # XXX
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory[idx] = val
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We can now write the first operand to memory and then read it back to the stack before operating on it.

In [20]:
mem = [0 for x in range(5)]

m = RWMem([
    PUSH(1),
    PUSH(2),
    # XXX Write operand to memory
    PUSH(3), PUSH(0), MSTORE(),
    # XXX Read operand from memory
    PUSH(0), MLOAD(),
    ADD(), 
    SUB(), 
], mem)

m.run()

4

To make memory writable, we need to use a different SMT solver primitive -- [arrays](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-arrays). Just like uninterpreted functions, arrays can be used to model mathematical functions, but arrays allow the function to be "updatable". The "update" does not modify the original function. It creates a new and updated function that has one input output pair changed. In our machine, on every write instruction, we replace the memory array with the new updated memory array. (TODO -- "I think update sounds like an in place change. I prefer to talk about it like an infinite map with the pure function store that takes an array and returns the modified array")

In [21]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ASSERT:
    n: int

class SymRWMem:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX
        self.memory = Array('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                # XXX
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory = Store(self.memory, idx, val)
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

We can now write a symbolic value to a concrete index for the first operand.

In [22]:
m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('x')), PUSH(0), MSTORE(),
    PUSH(0), MLOAD(),
    ADD(), 
    SUB(), 
    ASSERT(4)
])
m.run()

(Store(memory, 0, x)[0] + 2 - 1, 4, [x = 3])

We can even write a symbolic value to a symbolic index for the first operand. Note that the model solves for the symbolic value but does not have a value for the index. The model does not need to determine a concrete value for the index because for the model to be satisfiable, the operand just has to written to and read from the same index. The more formal term for the model "not having a value" is "does not have an interpretation".

In [23]:
idx = Int('idx')

m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('val')), PUSH(idx), MSTORE(),
    PUSH(idx), MLOAD(),
    ADD(), 
    SUB(),
    ASSERT(4)
])

m.run()

(Store(memory, idx, val)[idx] + 2 - 1, 4, [val = 3])

If we need a concrete value for the index, we can ask the model gives us a "default interpretation" for the expression by setting the `model_completion` flag to true.

In [24]:
idx = Int('idx')

m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('val')), PUSH(idx), MSTORE(),
    PUSH(idx), MLOAD(),
    ADD(), 
    SUB(),
    ASSERT(4)
])

(_, _, model) = m.run()

model.eval(idx, model_completion=True)

0

### Conditionals

Our next machine adds conditionals through the `ISZERO` instruction. `ISZERO`'s concrete implementation is straight forward.

In [25]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ISZERO:
    pass

class Cond:
    def __init__(self, code, memory=[]):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory[idx] = val
            case ISZERO():
                val = self.stack.pop()
                self.stack.append(1 if val == 0 else 0)
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

Let's modify our original example to check the expected answer in the machine.

In [26]:
# ((3 + 2) - 1)) - 4 == 0
m = Cond([
    PUSH(1), 
    PUSH(2), 
    PUSH(3), 
    ADD(), 
    SUB(),
    # XXX We expect our solution to be 4, try to zero out the solution
    PUSH(4),
    SUB(),
    ISZERO()
])
m.run()

1

There is an "if-then-else" SMT solver primitive. In Z3, this is the `If` function. Note, the Z3 `If` function represents the smtlib `ite` function (we'll get to smtlib later). The `If` function can be used to build a compound symbolic value. The compound symbolic value can be thought of as implicitly carrying a constraint.

In [27]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ISZERO:
    pass

@dataclass
class ASSERT:
    n: int

class SymCond:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = Array('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory = Store(self.memory, idx, val)
            case ISZERO():
                # XXX
                val = self.stack.pop()
                self.stack.append(If(val == 0, 1, 0))
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

We can now substitute a symbolic value into the operand and later assert the result of the `ISZERO` check. This later assertion can be thought of as setting that the implicit constraint in the `If` expression does hold.

In [28]:
m = SymCond([
    PUSH(1), 
    PUSH(2), 
    PUSH(Int('x')), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    # XXX
    ASSERT(1)
])
m.run()

(If(4 - (x + 2 - 1) == 0, 1, 0), 1, [x = 3])

We can instead assert that the constraint does not hold. Remember there are many valid concrete values `x` could resolve to in this case, but the constraint solver will just pick one for us.

In [29]:
m = SymCond([
    PUSH(1), 
    PUSH(2), 
    PUSH(Int('x')), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    # XXX
    ASSERT(0)
])
m.run()

(If(4 - (x + 2 - 1) == 0, 1, 0), 0, [x = 4])

### Conditional Jumps

### A more realistic model of memory using Bit Vectors